___

# [ Geoinformatika ] 
Katedra aplikované geoinformatiky a kartografie, Universita Karlova

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## GDAL-Python rastery


Tento notebook demonstruje zaklady prace s rastry v Python pomoci knihovny GDAL:  

* 1. Cteni rasteru do Numpy matice

* 2. Jednocucha mapova algebra 

* 3. Zapis rasteru na disk


### Priprava knihoven

In [4]:
import os
import numpy as np
from osgeo import gdal, gdal_array

# Cesta k adresari s daty!!! 
# PATH = './'
# PATH = '/mnt/home/k01/Downloads/geoinformatika-main/geodata'
PATH = '/Users/lukas/Work/prfuk/vyuka/geoinformatika/src/geoinformatika/geodata/'

In [5]:
! ls $PATH

geojson    geopackage gml        raster


In [6]:
! ls $PATH/raster

S2A_T33UVR_20180703T101029.tif ua18_cz_buildup.tif


### Rasterova data

In [7]:
raster_fn = os.path.join(PATH, 'raster', 'S2A_T33UVR_20180703T101029.tif')

In [8]:
raster_fn

'/Users/lukas/Work/prfuk/vyuka/geoinformatika/src/geoinformatika/geodata/raster/S2A_T33UVR_20180703T101029.tif'

### Cteni rasteru 

In [9]:
# ovladac / driver 
driver = gdal.GetDriverByName('Gtiff')

In [10]:
# otevreni 
ds = gdal.Open(raster_fn) 

In [11]:
# raster dimensions 
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print('{} sloupcu, {} radek a {} kanalu'.format(cols, rows, bands))

2292 sloupcu, 1364 radek a 12 kanalu


In [12]:
# geo meta
projection = ds.GetProjection()
geotransform = ds.GetGeoTransform()

In [13]:
projection

'PROJCS["WGS 84 / UTM zone 33N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32633"]]'

In [14]:
geotransform

(447370.0, 10.0, 0.0, 5554080.0, 0.0, -10.0)

In [15]:
# geotransformacni metadata
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

print('pocatek X: {}, pocatek Y: {} a velikost pixelu: {} m.'.format(originX, originY, pixelWidth))

pocatek X: 447370.0, pocatek Y: 5554080.0 a velikost pixelu: 10.0 m.


In [16]:
# nacteni vsech kanalu 
img = np.zeros((ds.RasterYSize, ds.RasterXSize, ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(ds.GetRasterBand(1).DataType))

for b in range(img.shape[2]):
    img[:, :, b] = ds.GetRasterBand(b + 1).ReadAsArray()

In [17]:
img.shape

(1364, 2292, 12)

In [18]:
img

array([[[14, 18, 24, ..., 42, 29, 29],
        [14, 19, 24, ..., 44, 30, 30],
        [14, 19, 24, ..., 44, 30, 30],
        ...,
        [ 5,  5,  3, ..., 14,  7,  7],
        [ 5,  6,  4, ..., 14,  7,  7],
        [ 5,  8,  4, ..., 19,  9,  9]],

       [[13, 18, 23, ..., 42, 29, 29],
        [13, 19, 24, ..., 44, 30, 30],
        [14, 19, 25, ..., 44, 30, 30],
        ...,
        [ 4,  5,  3, ..., 14,  7,  7],
        [ 4,  4,  3, ..., 14,  7,  7],
        [ 4,  5,  3, ..., 19,  9,  9]],

       [[14, 19, 24, ..., 43, 30, 30],
        [14, 19, 25, ..., 44, 30, 30],
        [14, 20, 26, ..., 44, 30, 30],
        ...,
        [ 5,  6,  4, ..., 26, 12, 12],
        [ 4,  7,  3, ..., 26, 12, 12],
        [ 4,  5,  2, ..., 24, 10, 10]],

       ...,

       [[13, 17, 23, ..., 49, 34, 34],
        [13, 18, 22, ..., 47, 32, 32],
        [13, 18, 22, ..., 47, 32, 32],
        ...,
        [28, 30, 32, ..., 56, 45, 45],
        [29, 31, 35, ..., 56, 45, 45],
        [19, 24, 25, ..., 67, 49

### Mapova algebra 

In [19]:
RED = 2
NIR = 3
index = img[:,:,NIR] + img[:,:,RED]

In [20]:
# ... jina analyza 
index

array([[55, 56, 56, ..., 10, 11, 13],
       [54, 56, 57, ..., 10, 10, 12],
       [57, 58, 59, ..., 15, 14, 10],
       ...,
       [53, 53, 53, ..., 65, 68, 59],
       [52, 54, 53, ..., 74, 72, 64],
       [52, 53, 53, ..., 69, 71, 71]], dtype=uint8)

### Zapis noveho raster

In [21]:
# priprava datove souboru, index.tif zmente na cokoliv jineho! 
jmeno_souboru = os.path.join(PATH, 'raster', 'index.tif')
vystup_ds = driver.Create(jmeno_souboru, ds.RasterXSize, ds.RasterYSize,  1, gdal.GDT_Int16)

In [22]:
# zapsani matice 
vystupni_kanal = vystup_ds.GetRasterBand(1).WriteArray(index)

In [23]:
vystupni_kanal

0

In [24]:
# zapis geo metadat
vystup_ds.SetProjection(projection)
vystup_ds.SetGeoTransform(geotransform)

0

In [ ]:
# vystup_ds.GetRasterBand(1).SetNoDataValue(10000)

In [25]:
# zapsani na disk 
vystup_ds = None

In [26]:
# uklid
img = None
ds = None

In [27]:
os.path.isfile(jmeno_souboru)

True